In [2]:
import pandas as pd
import re
import nltk
import spacy
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Asigură-te că stopwords și corpus-ul de cuvinte sunt descărcate
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')

# Încarcă modelul de limbă engleză din spacy
nlp = spacy.load("en_core_web_sm")

# Încarcă setul de date
data = pd.read_csv('balanced_lyrics_dataset.csv')

# Setează stopwords în engleză și lista de cuvinte în engleză
stop_words = set(stopwords.words('english'))
english_words = set(words.words())

def preprocess_text(text):
    # Păstrează doar caracterele alfabetice și convertește la lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    # Tokenizează textul
    words = word_tokenize(text)
    # Elimină stopwords și păstrează doar cuvintele în engleză
    words = [word for word in words if word not in stop_words and word in english_words]
    # Concatenează cuvintele pentru lemmatizare
    text = ' '.join(words)
    # Aplica lemmatizare folosind spacy
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc if token.lemma_ not in stop_words]
    return ' '.join(lemmatized_words)

# Aplică funcția de preprocesare pe coloana 'Cleaned_Lyrics'
data['Cleaned_Lyrics'] = data['Cleaned_Lyrics'].apply(preprocess_text)

# Salvează setul de date preprocesat
data.to_csv('preprocessed_lyrics_dataset.csv', index=False)

# Initializează vectorizatorul și modelul LDA
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
lda = LatentDirichletAllocation(n_components=10, random_state=0)

# Vectorizează textele din coloana 'Cleaned_Lyrics'
X = vectorizer.fit_transform(data['Cleaned_Lyrics'])

# Antrenează modelul LDA pe datele vectorizate
lda.fit(X)

# Obține distribuția topic-urilor pentru fiecare instanță de versuri
topic_distributions = lda.transform(X)

# Identifică topic-ul dominant pentru fiecare instanță
dominant_topics = topic_distributions.argmax(axis=1)

# Adaugă topic-ul dominant la setul de date original
data['topic_dominant'] = dominant_topics

# Funcție pentru a obține cuvintele cele mai relevante pentru fiecare topic
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words[topic_idx] = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    return top_words

# Obține cuvintele corespunzătoare fiecărui topic
feature_names = vectorizer.get_feature_names_out()
top_words = get_top_words(lda, feature_names, 20)  # 20 de cuvinte pentru fiecare topic

# Convert the dictionary to a DataFrame
df_top_words = pd.DataFrame(dict([(f'Topic {i}', top_words[i]) for i in top_words]))

# Salvează cuvintele predominante într-un fișier CSV
df_top_words.to_csv('nou_top_words_per_topic.csv', index=False)

# Afișează cuvintele pentru fiecare topic
for topic, words in top_words.items():
    print(f"Topic {topic}: {', '.join(words)}")

# Analiza frecvenței topicurilor pe genuri
topic_genre_distribution = data.groupby(['Genre', 'topic_dominant']).size().unstack(fill_value=0)

# Afișează distribuția topicurilor pe genuri
print(topic_genre_distribution)

# Salvează distribuția topicurilor pe genuri într-un fișier CSV
topic_genre_distribution.to_csv('nou_topic_genre_distribution.csv')

# Salvează datele modificate într-un fișier CSV
data.to_csv('nou_dataset_with_dominant_topics.csv', index=False)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Topic 0: love, heart, know, night, say, like, baby, make, feel, oh, come, kiss, look, blue, ill, dream, day, tell, time, song
Topic 1: world, fly, burn, pretty, young, star, sky, hand, night, life, time, tonight, rise, reach, revolution, alright, chorus, come, drunk, like
Topic 2: na, like, wan, em, come, know, cause, verse, man, ya, make, little, let, think, hook, bone, hard, beat, say, die
Topic 3: know, oh, say, yeah, want, baby, time, let, come, way, cause, like, tell, make, feel, girl, need, right, ill, think
Topic 4: time, die, away, come, fall, end, lose, world, life, break, leave, blood, hell, live, stand, light, inside, war, fight, feel
Topic 5: day, life, night, come, way, ill, know, turn, oh, rain, time, sleep, wait, live, away, make, like, pain, man, think
Topic 6: bitch, like, ya, money, ass, yeah, know, verse, man, come, roll, say, da, dick, cause, em, pussy, yo, ai, chorus
Topic 7: like, say, tell, cause, verse, yeah, know, live, time, mind, people, em, make, high, think